In [ ]:
import ee
import geopandas as gpd
import fiona
import shapely
import contextily as cx
import json
from src.visualization import ee_viz
from src.data import mtbs
from src.data import shape_processor
import src.data.ee_utils as ee_utils
import numpy as np

import importlib
importlib.reload(ee_viz)
importlib.reload(ee_utils)
importlib.reload(mtbs)

In [ ]:
ee.Initialize()

### Get Regions of interest

In [ ]:
# Fetch simplified regions of interest - to avoid processing problems with really complicated shapes.
seki = gpd.read_file("../../data/shapefiles/seki_convex_hull.shp")
sierras = gpd.read_file("../../data/shapefiles/sierras_convex_hull.shp")

In [ ]:
seki_complex = gpd.read_file("../../data/shapefiles/seki.shp")
seki_box = shape_processor.get_box(seki_complex)

In [ ]:
sierras_complex = gpd.read_file("../../data/shapefiles/sierras.shp")
sierras_box = shape_processor.get_box(sierras_complex)

In [ ]:
seki.to_crs(3857).explore()

In [ ]:
sierras.to_crs(3857).explore()

### Get Burn Severity Data

In [ ]:
burn_severity_ic = mtbs.get_burn_severity_data().sort('system:time_start', False)

In [ ]:
ee_viz.viz_burn_severity(burn_severity_ic.first(), seki.geometry.iloc[0], ['burn_severity', 'burn_year'])

### Get aggregated burn data - last burn severity for each pixel, and burn count for each pixel

In [ ]:
burn_data = mtbs.get_aggregated_burn_data()
burn_data

In [ ]:
ee_viz.viz_burn_severity(burn_data, seki.geometry.iloc[0], ['last_burn_severity', 'last_burn_year', 'burn_count'])

In [ ]:
ee_viz.viz_burn_severity(burn_data, sierras.geometry.iloc[0], ['last_burn_severity', 'last_burn_year', 'burn_count'])

### Save Fire Raster as Geo Tiff

In [ ]:
ee_utils.save_image_to_drive(burn_data, polygon=seki_box.geometry[0], img_name='burn_data', scale=30)

In [ ]:
ee_utils.save_image_to_drive(burn_data, polygon=sierras_box.geometry[0], img_name='burn_data_sierras', scale=30)